<a href="https://colab.research.google.com/github/raleight1/MGMT467_Team2/blob/main/Final%20Project/Individual%20Notebooks/Final_Ryan_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
from google.cloud import bigquery

# Redefine constants (as they were not preserved in the previous execution context)
PROJECT_ID = "mgmt467-project"
DATASET_ID = "GymDB"
MODEL_NAME = "churn_prediction_model"

bq = bigquery.Client(project=PROJECT_ID)

# Re-authenticate to ensure fresh credentials
auth.authenticate_user()
print("✅ Re-authenticated to Google Cloud")

evaluation_query = f"""
SELECT
  *
FROM
  ML.EVALUATE(MODEL `{PROJECT_ID}.{DATASET_ID}.{MODEL_NAME}`)
"""

print("Running model evaluation query...")
eval_result = bq.query(evaluation_query).result().to_dataframe()

feature_importance_query = f"""
SELECT
  * EXCEPT(category_weights)
FROM
  ML.WEIGHTS(MODEL `{PROJECT_ID}.{DATASET_ID}.{MODEL_NAME}`)
ORDER BY
  ABS(weight) DESC
"""

print("Running feature importance query...")
feature_importance_result = bq.query(feature_importance_query).result().to_dataframe()

model_metrics = eval_result.iloc[0]
feature_importances = feature_importance_result.copy()

# Identify top influential features (excluding intercept)
top_features = feature_importances[feature_importances['processed_input'] != '__INTERCEPT__'] \
    .sort_values(by='weight', key=abs, ascending=False)

# Extract weather-related features
weather_features = top_features[
    top_features['processed_input'].isin([
        'avg_daily_temperature_c',
        'avg_daily_wind_speed_kph',
        'avg_daily_relative_humidity',
        'day_proportion'
    ])
]

print("Model metrics:")
print(model_metrics)
print("\nTop features (excluding intercept) by absolute weight:")
print(top_features)
print("\nWeather-related feature importances:")
print(weather_features)

✅ Re-authenticated to Google Cloud
Running model evaluation query...
Running feature importance query...
Model metrics:
precision    1.000000
recall       0.025641
accuracy     0.746667
f1_score     0.050000
log_loss     0.554323
roc_auc      0.595709
Name: 0, dtype: float64

Top features (excluding intercept) by absolute weight:
               processed_input    weight
1      avg_daily_temperature_c -0.061587
2                          Age  0.012526
3     avg_daily_wind_speed_kph -0.002902
4  avg_daily_relative_humidity -0.001968
5                  Tenure_Days -0.000028
6               day_proportion  0.000000
7                       Gender       NaN
8              Membership_Type       NaN

Weather-related feature importances:
               processed_input    weight
1      avg_daily_temperature_c -0.061587
3     avg_daily_wind_speed_kph -0.002902
4  avg_daily_relative_humidity -0.001968
6               day_proportion  0.000000


Create a comprehensive DIVE analysis for the trained BQML churn prediction model, including a 'Data' section summarizing the GymData_Curated and Weather_Raw_Streaming datasets and features, an 'Insights' section detailing key findings from ML.EVALUATE and ML.WEIGHTS with a focus on weather features, and an 'Evaluation & Next Steps' section outlining model performance, limitations (e.g., low recall), and future improvements. Conclude with one substantive question the model can address.


# Task
Create a comprehensive DIVE analysis for the `churn_prediction_model` in the `GymDB` dataset. The analysis should include:
*   A 'Data' section summarizing the `GymData_Curated` and `Weather_Raw_Streaming` datasets and the features they contribute to the model.
*   An 'Insights' section detailing key findings from the `ML.EVALUATE` results (`model_metrics` and `eval_result`), including metrics like precision, recall, accuracy, and ROC AUC. This section should also analyze the `ML.WEIGHTS` results (`feature_importance_result`, `top_features`, and `weather_features`), highlighting overall influential features and specifically focusing on the importance and impact of weather-related features.
*   An 'Evaluation & Next Steps' section assessing the model's overall performance, discussing its limitations (e.g., the observed low recall), and suggesting concrete future improvements.
*   Conclude with one substantive question that the churn prediction model can effectively address.
Integrate all generated sections into a comprehensive DIVE analysis document.

## Data Section: Summarize Datasets and Features

### Subtask:
Create the 'Data' section of the DIVE analysis, summarizing the `GymData_Curated` and `Weather_Raw_Streaming` datasets and the features they contribute to the `churn_prediction_model`.


## Data

This section describes the datasets used to train the churn prediction model and the features extracted from them.

### GymData_Curated Dataset
This dataset provides information about individual gym members and their membership details. The features contributed by this dataset to the model include:

*   `Age`: Represents the age of the gym member, which can influence their likelihood of churning.
*   `Tenure_Days`: Indicates the duration, in days, that a member has been subscribed to the gym. Longer tenure might correlate with lower churn rates.
*   `Gender`: The gender of the gym member, potentially revealing different churn patterns across demographics.
*   `Membership_Type`: The type of membership held by the member (e.g., monthly, annual, premium), which could be a significant factor in predicting churn due to varying commitment levels or benefits.

### Weather_Raw_Streaming Dataset
This dataset contains real-time and historical weather information, which has been integrated to analyze its impact on gym member churn. The weather-related features used in the model are:

*   `avg_daily_temperature_c`: The average daily temperature in Celsius. Extreme temperatures (hot or cold) might deter members from visiting the gym.
*   `avg_daily_wind_speed_kph`: The average daily wind speed in kilometers per hour. High winds could make travel to the gym less appealing.
*   `avg_daily_relative_humidity`: The average daily relative humidity. High humidity can affect comfort levels for outdoor activities or travel.
*   `day_proportion`: A feature likely representing the proportion of daylight or a categorical representation of the time of day, which could influence when members are most likely to visit or churn.

## Insights Section: Model Evaluation and Feature Importance

### Subtask:
Develop the 'Insights' section of the DIVE analysis, detailing key findings from the ML.EVALUATE and ML.WEIGHTS results, with a focus on overall influential features and specifically on weather-related features.


## Insights

### Model Evaluation Metrics

The model's performance was evaluated using standard classification metrics. Here are the key findings:

*   **Precision**: `1.00` - This indicates that when the model predicts a customer will churn, it is always correct. This is a very high precision, suggesting that false positives (predicting churn when a customer does not churn) are very low or non-existent.
*   **Recall**: `0.0256` - This metric is quite low, meaning the model only identified a very small percentage (about 2.56%) of the actual churners. While it's accurate when it predicts churn, it misses a significant number of customers who *do* churn.
*   **Accuracy**: `0.7467` - The overall accuracy of the model is approximately 74.67%. This represents the proportion of total predictions that were correct (both churners and non-churners).
*   **F1-Score**: `0.0500` - The F1-score is the harmonic mean of precision and recall. A low F1-score indicates an imbalance between precision and recall, as seen with our very high precision but very low recall.
*   **Log Loss**: `0.5543` - Log loss is a measure of the uncertainty of the predictions. Lower values are better. A log loss of 0.5543 suggests there is still a notable amount of uncertainty in the model's predictions.
*   **ROC AUC**: `0.5957` - The Area Under the Receiver Operating Characteristic Curve (ROC AUC) measures the model's ability to distinguish between churners and non-churners. A value of 0.5957 is slightly better than random guessing (0.5), but it indicates the model has limited discriminatory power.

**Summary**: The model exhibits extremely high precision, meaning its positive predictions are very reliable. However, its very low recall suggests it struggles to identify most of the actual churn cases. This behavior might be due to an imbalanced dataset where churn events are rare, leading the model to be conservative in its churn predictions. The moderate accuracy and low ROC AUC further suggest that while the model is precise, it's not very robust in capturing the overall churn phenomenon.

### Feature Importance

Feature importance was determined by the absolute weights assigned to each feature by the model. The weights indicate the strength and direction of a feature's influence on the likelihood of churn.

**Overall Most Influential Features (excluding intercept) by Absolute Weight:**

*   **avg_daily_temperature_c** (weight: `-0.061587`): This is the most influential feature by a significant margin. The negative weight indicates an inverse relationship with churn; as the average daily temperature increases, the likelihood of customer churn decreases. This might suggest that people are less likely to cancel gym memberships in warmer weather, perhaps due to increased outdoor activity or motivation.
*   **Age** (weight: `0.012526`): With a positive weight, an increase in age is associated with a higher likelihood of churn. This could imply that older members are more prone to churning, possibly due to changing fitness needs or activity levels.
*   **avg_daily_wind_speed_kph** (weight: `-0.002902`): The negative weight suggests that higher average daily wind speed slightly decreases the likelihood of churn. This influence is much smaller than temperature.

Other features like `avg_daily_relative_humidity`, `Tenure_Days`, and `day_proportion` have smaller absolute weights, indicating a lesser impact on churn predictions.

#### Weather-Related Feature Importances

Focusing specifically on weather-related features:

*   **avg_daily_temperature_c** (weight: `-0.061587`): As previously noted, this feature has the strongest influence among all features. A higher average daily temperature significantly *reduces* the probability of churn.
*   **avg_daily_wind_speed_kph** (weight: `-0.002902`): This feature has a minor negative impact on churn. Higher wind speeds are associated with a very slight decrease in churn probability.
*   **avg_daily_relative_humidity** (weight: `-0.001968`): Similarly, higher average daily relative humidity is associated with a very slight decrease in churn probability, though its impact is minimal.
*   **day_proportion** (weight: `0.000000`): This feature has a weight of 0, indicating it had virtually no influence on the churn prediction in this model. It's possible that this feature was deemed irrelevant or had no predictive power by the model.

**Features with NaN Weights**: Features such as `Gender` and `Membership_Type` show `NaN` weights. This typically occurs for categorical features when the model (specifically, BigQuery ML's linear/logistic regression models) determines that these features have no significant statistical relationship to the target variable after one-hot encoding or when all categories have been dropped due to insufficient variance or correlation, or if they were effectively excluded during model training due to their lack of predictive power. In simpler terms, these features were not found to be useful in predicting churn by the current model.

## Insights

### Model Evaluation Metrics

The model's performance was evaluated using standard classification metrics. Here are the key findings:

*   **Precision**: `1.00` - This indicates that when the model predicts a customer will churn, it is always correct. This is a very high precision, suggesting that false positives (predicting churn when a customer does not churn) are very low or non-existent.
*   **Recall**: `0.0256` - This metric is quite low, meaning the model only identified a very small percentage (about 2.56%) of the actual churners. While it's accurate when it predicts churn, it misses a significant number of customers who *do* churn.
*   **Accuracy**: `0.7467` - The overall accuracy of the model is approximately 74.67%. This represents the proportion of total predictions that were correct (both churners and non-churners).
*   **F1-Score**: `0.0500` - The F1-score is the harmonic mean of precision and recall. A low F1-score indicates an imbalance between precision and recall, as seen with our very high precision but very low recall.
*   **Log Loss**: `0.5543` - Log loss is a measure of the uncertainty of the predictions. Lower values are better. A log loss of 0.5543 suggests there is still a notable amount of uncertainty in the model's predictions.
*   **ROC AUC**: `0.5957` - The Area Under the Receiver Operating Characteristic Curve (ROC AUC) measures the model's ability to distinguish between churners and non-churners. A value of 0.5957 is slightly better than random guessing (0.5), but it indicates the model has limited discriminatory power.

**Summary**: The model exhibits extremely high precision, meaning its positive predictions are very reliable. However, its very low recall suggests it struggles to identify most of the actual churn cases. This behavior might be due to an imbalanced dataset where churn events are rare, leading the model to be conservative in its churn predictions. The moderate accuracy and low ROC AUC further suggest that while the model is precise, it's not very robust in capturing the overall churn phenomenon.

### Feature Importance

Feature importance was determined by the absolute weights assigned to each feature by the model. The weights indicate the strength and direction of a feature's influence on the likelihood of churn.

**Overall Most Influential Features (excluding intercept) by Absolute Weight:**

*   **avg_daily_temperature_c** (weight: `-0.061587`): This is the most influential feature by a significant margin. The negative weight indicates an inverse relationship with churn; as the average daily temperature increases, the likelihood of customer churn decreases. This might suggest that people are less likely to cancel gym memberships in warmer weather, perhaps due to increased outdoor activity or motivation.
*   **Age** (weight: `0.012526`): With a positive weight, an increase in age is associated with a higher likelihood of churn. This could imply that older members are more prone to churning, possibly due to changing fitness needs or activity levels.
*   **avg_daily_wind_speed_kph** (weight: `-0.002902`): The negative weight suggests that higher average daily wind speed slightly decreases the likelihood of churn. This influence is much smaller than temperature.

Other features like `avg_daily_relative_humidity`, `Tenure_Days`, and `day_proportion` have smaller absolute weights, indicating a lesser impact on churn predictions.

#### Weather-Related Feature Importances

Focusing specifically on weather-related features:

*   **avg_daily_temperature_c** (weight: `-0.061587`): As previously noted, this feature has the strongest influence among all features. A higher average daily temperature significantly *reduces* the probability of churn.
*   **avg_daily_wind_speed_kph** (weight: `-0.002902`): This feature has a minor negative impact on churn. Higher wind speeds are associated with a very slight decrease in churn probability.
*   **avg_daily_relative_humidity** (weight: `-0.001968`): Similarly, higher average daily relative humidity is associated with a very slight decrease in churn probability, though its impact is minimal.
*   **day_proportion** (weight: `0.000000`): This feature has a weight of 0, indicating it had virtually no influence on the churn prediction in this model. It's possible that this feature was deemed irrelevant or had no predictive power by the model.

**Features with NaN Weights**: Features such as `Gender` and `Membership_Type` show `NaN` weights. This typically occurs for categorical features when the model (specifically, BigQuery ML's linear/logistic regression models) determines that these features have no significant statistical relationship to the target variable after one-hot encoding or when all categories have been dropped due to insufficient variance or correlation, or if they were effectively excluded during model training due to their lack of predictive power. In simpler terms, these features were not found to be useful in predicting churn by the current model.

## Evaluation & Next Steps Section: Performance, Limitations, and Improvements

### Subtask:
Construct the 'Evaluation & Next Steps' section of the DIVE analysis, including an assessment of the model's overall performance, a discussion of its limitations (e.g., the observed low recall), and concrete suggestions for future improvements to address these limitations and enhance the model.


## Evaluation & Next Steps

### Model Performance Assessment
The `churn_prediction_model` demonstrates a mixed performance profile. It achieves a perfect `precision` of `1.00`, indicating that when the model predicts a customer will churn, it is always correct. This is a strong point for scenarios where false positives are costly. However, this comes at the significant cost of an extremely low `recall` of `0.0256`. This means the model only identifies about 2.56% of actual churners. The `accuracy` is `0.7467`, which appears decent but can be misleading in imbalanced datasets, as it might primarily reflect the model's ability to correctly classify the majority non-churning class. The `f1_score` is `0.0500`, a harmonic mean of precision and recall, which is very low due to the poor recall. The `roc_auc` score is `0.5957`, suggesting that the model has some discriminatory power, but it's only slightly better than random guessing (0.5).

### Limitations
The primary limitation of the current model is its very low `recall`. While its perfect precision means no false positives, the model fails to identify the vast majority of customers who actually churn. This means that if the goal is to proactively intervene with potential churners, this model would miss almost all of them, rendering it largely ineffective for retention strategies. Other limitations include:

*   **Moderate `ROC AUC`**: A score of `0.5957` suggests limited ability to distinguish between churning and non-churning customers.
*   **Unutilized Categorical Features**: The `NaN` weights for `Gender` and `Membership_Type` indicate these potentially important features were not effectively utilized in the model. This could be due to issues in data preprocessing, feature encoding, or a lack of variance within these features that prevents the model from learning meaningful weights.

### Future Improvements
To address the identified limitations and enhance the model's performance, especially its ability to detect churners, the following improvements are suggested:

*   **Addressing Data Imbalance**: The extremely low recall strongly suggests a severe class imbalance problem (likely many more non-churners than churners). Techniques to mitigate this should be explored:
    *   **Oversampling** the minority class (churners) using methods like `SMOTE`.
    *   **Undersampling** the majority class (non-churners).
    *   Adjusting **class weights** during model training to penalize misclassifications of the minority class more heavily.
*   **Feature Engineering**: Explore creating new features or refining existing ones to provide the model with more predictive power:
    *   **Interaction terms** between existing features.
    *   **Polynomial features** for non-linear relationships.
    *   Incorporating more granular **membership activity data** (e.g., frequency of visits, types of classes attended, recent attendance trends).
*   **Exploring Different Model Architectures**: Linear models like logistic regression (which BQML often uses for classification) might not be sufficient for complex churn prediction. Consider alternative machine learning algorithms that can handle imbalanced data better or capture more complex relationships:
    *   **Gradient Boosting Machines** (e.g., XGBoost, LightGBM).
    *   **Neural Networks**.
*   **Investigating Current Features**: A detailed investigation into why `Gender` and `Membership_Type` resulted in `NaN` weights is crucial. This could involve:
    *   Checking for **data quality issues**.
    *   Experimenting with different **encoding strategies** (e.g., one-hot encoding for categorical features).
    *   Verifying if these features truly have predictive power in the dataset.
*   **Business Goal Alignment**: Re-evaluate the model's objective based on business needs. If identifying as many churners as possible is critical, even with a slightly higher false positive rate, then optimizing for `recall` or `F1-score` over `precision` would be more appropriate. If the goal is highly confident predictions for a targeted marketing campaign, the current high precision might be desirable, but the low recall would still limit its overall impact.



**Substantive Question:** Given the significant negative association between average daily temperature and churn, coupled with the influence of member age, and the model's high precision in identifying a subset of churners, how can the gym proactively utilize weather forecasts and member age data to target high-risk demographics with personalized retention incentives during predicted periods of unfavorable weather conditions?

## Propose Dashboard Integration

### Subtask:
Outline how the generated insights and interactive Plotly visualization could be integrated into a dashboard section, explaining their influence and value for a larger reporting context.

### Dashboard Integration
The interactive Plotly visualization of feature importances, along with the model evaluation metrics, can be integrated into a dedicated 'Churn Prediction Model Performance' dashboard. This dashboard would serve as a central hub for stakeholders to monitor the model's effectiveness and understand the driving factors behind churn.

**Proposed Dashboard Section: 'Churn Driver Analysis'**

1.  **Interactive Feature Importance Chart**: The Plotly bar chart displaying `ML.WEIGHTS` results (`feature_importances`) should be a key component. This allows users to:
    *   **Visually identify top churn drivers**: Quickly see which features (e.g., `avg_daily_temperature_c`, `Age`) have the most significant impact.
    *   **Understand direction of influence**: The positive/negative weights provide immediate insight into how each feature affects churn probability.
    *   **Drill down into specifics**: Hover-over functionality can display exact weight values and feature names for detailed understanding.

2.  **Model Performance Metrics (Overview)**: Key metrics from `ML.EVALUATE` (`precision`, `recall`, `accuracy`, `roc_auc`, `f1_score`) should be displayed prominently, perhaps as gauge charts or scorecards. This provides a quick snapshot of the model's overall health.

3.  **Weather Impact Trends**: Given the strong influence of `avg_daily_temperature_c` and other weather features, a dedicated section showing historical weather trends overlaid with churn rates could be valuable. This could be another interactive plot, allowing filtering by location or time period.

**Value for Stakeholders**: This dashboard section would empower business users to:
*   **Prioritize interventions**: Focus on influencing the most impactful churn drivers.
*   **Tailor retention strategies**: Develop weather-sensitive campaigns or age-specific offers.
*   **Monitor model stability**: Track how feature importances or model performance metrics change over time, indicating potential data shifts or model degradation.

**Link to Dashboard Section**: This specific 'Churn Driver Analysis' section would be a crucial part of a broader 'Customer Retention Dashboard', providing actionable insights derived directly from the model's mechanics.

In [2]:
import plotly.express as px

# Create an interactive bar chart for feature importances
fig = px.bar(
    feature_importances.sort_values(by='weight', key=abs, ascending=False),
    x='processed_input',
    y='weight',
    title='Interactive Feature Importance for Churn Prediction Model',
    labels={'processed_input': 'Feature', 'weight': 'Weight'},
    hover_data=['processed_input', 'weight']
)

fig.update_layout(
    xaxis_title='Feature',
    yaxis_title='Weight (Impact on Churn)',
    xaxis_tickangle=-45,
    height=500,
    width=800
)

fig.show()

### Interactive Plotly Visualization: Feature Importance

This interactive bar chart illustrates the absolute weights of each feature in the churn prediction model. Features with larger absolute weights have a greater impact on the model's prediction. The chart allows for easy comparison of feature influence and hovering over bars reveals exact weight values.

Key observations from this visualization reinforce the earlier findings: `avg_daily_temperature_c` stands out as the most influential feature, followed by `Age`. The relative impact of other features like `avg_daily_wind_speed_kph` and `avg_daily_relative_humidity` is also clearly visible.

## Compile DIVE Analysis with Prompt Logs

### Subtask:
Integrate all the generated sections ('Data', 'Insights', 'Evaluation & Next Steps', and the substantive question) into a comprehensive DIVE analysis document.


## DIVE Analysis: Gym Churn Prediction Model

This document presents a comprehensive DIVE analysis for the BigQuery ML churn prediction model, integrating data descriptions, key insights from model evaluation and feature importance, and outlining next steps for improvement.

### 1. Data

The churn prediction model leverages two primary datasets:

*   **`GymDB.GymData_Curated`**: This dataset likely contains customer-specific information relevant to gym membership, such as `Age`, `Tenure_Days`, `Gender`, and `Membership_Type`.
*   **`GymDB.Weather_Raw_Streaming`**: This dataset is integrated to provide environmental context, including features like `avg_daily_temperature_c`, `avg_daily_wind_speed_kph`, `avg_daily_relative_humidity`, and `day_proportion`.

These datasets are combined to provide a holistic view of factors influencing customer churn for the model named `churn_prediction_model` within the `mgmt467-project`.

### 2. Insights

**Model Evaluation (`ML.EVALUATE`)**: The initial evaluation of the model (`ML.EVALUATE`) provides the following key metrics:

```
precision    1.000000
recall       0.025641
accuracy     0.746667
f1_score     0.050000
log_loss     0.554323
roc_auc      0.595709
```

**Feature Importance (`ML.WEIGHTS`)**: Analyzing the feature weights reveals the most influential factors in predicting churn:

| processed_input             | weight      |
| :-------------------------- | :---------- |
| `avg_daily_temperature_c`   | -0.061587   |
| `Age`                       | 0.012526    |
| `avg_daily_wind_speed_kph`  | -0.002902   |
| `avg_daily_relative_humidity` | -0.001968   |
| `Tenure_Days`               | -0.000028   |
| `day_proportion`            | 0.000000    |
| `Gender`                    | NaN         |
| `Membership_Type`           | NaN         |

**Key Findings**:
*   **Temperature as a Strong Predictor**: `avg_daily_temperature_c` is the most influential feature by absolute weight, indicating a strong negative correlation with churn (lower temperatures associated with higher churn, or vice-versa depending on the true meaning of the negative weight).
*   **Age's Influence**: `Age` also shows a notable positive weight, suggesting older members might be slightly more prone to churn or that it's a factor in churn prediction.
*   **Weather Impact**: Other weather-related features like `avg_daily_wind_speed_kph` and `avg_daily_relative_humidity` have smaller but present weights, suggesting that weather conditions collectively play a role in churn behavior. `day_proportion` has a weight of 0, implying it had no predictive power in this model.
*   **Categorical Features**: `Gender` and `Membership_Type` show `NaN` weights, which might indicate that they were either dropped during preprocessing, had zero variance, or their categories did not contribute meaningfully to the linear model.

### 3. Evaluation & Next Steps

**Model Performance**: The model exhibits a very high precision (1.0), meaning that when it predicts churn, it is always correct. However, its recall is extremely low (0.0256), indicating that it misses most of the actual churners. The accuracy is moderate (0.746), but given the imbalanced recall, this might be misleading. The `f1_score` (0.05) and `roc_auc` (0.595) further confirm that while the model is precise in its few positive predictions, it is generally poor at identifying churners.

**Limitations**: The most significant limitation is the very low recall. This suggests the model is highly conservative in predicting churn, likely due to class imbalance or insufficient predictive power for the positive class. The `NaN` weights for categorical features also suggest potential issues with how these features were handled or their lack of predictive value.

**Future Improvements**:
*   **Address Class Imbalance**: Techniques like oversampling the minority class (churners), undersampling the majority class (non-churners), or using cost-sensitive learning should be explored.
*   **Feature Engineering**: Revisit and potentially enhance features from `Gender` and `Membership_Type` or explore interaction terms.
*   **Model Selection**: Consider alternative models better suited for imbalanced datasets or non-linear relationships, such as Gradient Boosting Machines (XGBoost, LightGBM) or neural networks.
*   **Hyperparameter Tuning**: Optimize the model's hyperparameters to improve recall without severely sacrificing precision.

### 4. Substantive Question

Given the observed influence of weather on churn, **'How does extreme weather (e.g., unusually hot/cold days, high winds) impact a customer's likelihood of churning within a specific time frame, and what targeted interventions could mitigate this effect?'**

## Final Task

### Subtask:
Review the complete DIVE analysis to ensure all requirements are met, including the summary of datasets, detailed insights from model evaluation and feature importance, discussion of limitations and improvements, and the substantive question.


## Summary:

### Data Analysis Key Findings

*   **Data Sources and Features**: The churn prediction model utilized two datasets:
    *   `GymData_Curated`: Contributed member-specific features such as `Age`, `Tenure_Days`, `Gender`, and `Membership_Type`.
    *   `Weather_Raw_Streaming`: Provided environmental context through features like `avg_daily_temperature_c`, `avg_daily_wind_speed_kph`, `avg_daily_relative_humidity`, and `day_proportion`.
*   **Model Performance Overview**: The model demonstrated a mixed performance profile with a perfect `precision` of `1.00`, but an extremely low `recall` of `0.0256`. The overall `accuracy` was `0.7467`, the `f1_score` was `0.0500`, `log_loss` was `0.5543`, and `roc_auc` was `0.5957`.
*   **High Precision, Low Recall**: While the model is always correct when it predicts a customer will churn, it only identifies approximately 2.56% of actual churners, indicating it misses the vast majority of churn events.
*   **Feature Importance**:
    *   `avg_daily_temperature_c` was identified as the most influential feature (weight: `-0.061587`), showing a strong inverse relationship with churn (higher temperatures associated with lower churn probability).
    *   `Age` was also a notable influencer (weight: `0.012526`), suggesting older members might be more prone to churn.
    *   Other weather features like `avg_daily_wind_speed_kph` (weight: `-0.002902`) and `avg_daily_relative_humidity` (weight: `-0.001968`) had minor impacts.
    *   `day_proportion` had no observed influence (weight: `0.000000`).
*   **Unutilized Categorical Features**: `Gender` and `Membership_Type` exhibited `NaN` weights, implying they were not effectively utilized by the model, potentially due to preprocessing issues, lack of variance, or insufficient predictive power.

### Insights or Next Steps

*   **Address Class Imbalance**: The model's low recall is a critical limitation for proactive churn intervention. Future efforts should focus on addressing the likely class imbalance using techniques such as oversampling (e.g., SMOTE), undersampling, or adjusting class weights during training.
*   **Strategic Interventions and Further Investigation**: The gym should investigate how to proactively utilize weather forecasts and member age data to target high-risk demographics with personalized retention incentives, particularly during predicted periods of unfavorable weather conditions. For example, "How does extreme weather (e.g., unusually hot/cold days, high winds) impact a customer's likelihood of churning within a specific time frame, and what targeted interventions could mitigate this effect?"
